In [8]:
# -*- coding: utf-8 -*-
"""
Created on Fri Feb  7 14:54:32 2025

Refer to the following webpage:
    https://biosteam.readthedocs.io/en/latest/tutorial/Glacial_acetic_acid_separation.html
    
@author: k2473520
"""

import biosteam as bst
import pdb
import numpy as np
from scipy.optimize import differential_evolution # population based optimization code
from scipy.stats import qmc # latin hypercube sampling 



ModuleNotFoundError: No module named 'biosteam'

In [10]:
pip install --user --ignore-installed biosteam

  Using cached biosteam-2.50.4-py3-none-any.whl
  Using cached thermosteam-0.50.2-py3-none-any.whl
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached chaospy-4.3.17-py3-none-any.whl.metadata (5.4 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached numpy-2.2.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached numpoly-1.3.6.tar.gz (369 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached decorator-5.1.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached jedi-0.19.2-py2.py3-none-any.whl.meta

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [170 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build\lib.win-amd64-cpython-312\numpoly
  copying numpoly\align.py -> build\lib.win-amd64-cpython-312\numpoly
  copying numpoly\baseclass.py -> build\lib.win-amd64-cpython-312\numpoly
  copying numpoly\dispatch.py -> build\lib.win-amd64-cpython-312\numpoly
  copying numpoly\option.py -> build\lib.win-amd64-cpython-312\numpoly
  copying numpoly\sympy_.py -> build\lib.win-amd64-cpython-312\numpoly
  copying numpoly\__init__.py -> build\lib.win-amd64-cpython-312\numpoly
  creating build\lib.win-amd64-cpython-312\numpoly\array_function
  copying numpoly\array_function\absolute.py -> build\lib.win-amd64-cpython-312\numpoly\array_function
  copying numpoly\array_function\add.py -> build\lib.win-amd64-cpython-312\numpoly\array_function
  copying numpoly\array_function\all.py -> build\lib.win-amd64-cpython-312\numpoly\array_function
  copying nu

In [ ]:

class BlackBox():
    
    def __init__(self, verbose=False):
        bst.nbtutorial() # For light-mode diagrams, ignore warnings
        self.set_base_model()
        self.nEval = 0
        self.verbose = verbose
        print(" ##### An instance of the 'BlackBox' class  has been initialised!")
    
    def set_base_model(self):
        # relevant values based on the website example
        # _n1 = 12 # number of stages for extractor
        # _Lr1, _Hr1, _k1  = [0.95, 0.95, 1.4]
        # _Lr2, _Hr2, _k2  = [0.999, 0.999, 1.4]
        # _T_hex = 310
        # _Lr3, _Hr3, _k3  = [0.99, 0.99, 1.5]
        X = [12, 0.95, 0.95, 1.4, 0.999, 0.999, 1.4, 310, 0.99, 0.99, 1.5]
        self._set(X)
        
    def _bounds(self):
        # bounds for variables. feel free to change!
        bounds = [(0, 50), # no. of stages in extractor
                  (0, 0.9999), (0, 0.9999),  # light key, heavy key, 'k' for extract distiller
                  (0, 0.9999), (0, 0.9999),  # light key, heavy key, 'k' for acetic_acid_purification
                  (273, 350), # temperature for 'HX'
                  (0, 0.9999), (0, 0.9999),  # light key, heavy key, 'k' for reffinate_distiller
                  ]
        return bounds
    
    def _integrality(self):
        # which varibales are integers (True if integer)
        ints = [True, 
                False, False, False,
                False, False, False,
                False, 
                False, False, False,
                ]
        return ints
        
        
    def _set(self, X):
        
        # Define chemicals used in the process
        bst.settings.set_thermo(['Water', 'AceticAcid', 'EthylAcetate'])

        _n1 = X[0] # number of stages for extractor
        _Lr1, _Hr1,   = X[1:3]
        _Lr2, _Hr2,   = X[3:5]
        _T_hex = X[5]
        _Lr3, _Hr3  = X[6:8]
        
        # ensure that integer variables are in fact integer
        _n1 = int(_n1)
        
        
        
        # Amount of ethyl-acetate to fermentation broth
        solvent_feed_ratio = 1.5

        # Fermentation broth with dilute acetic acid
        acetic_acid_broth = bst.Stream(ID='acetic_acid_broth', AceticAcid=1000, Water=9000, units='kg/hr')

        # Solvent
        ethyl_acetate = bst.Stream(ID='ethyl_acetate',  EthylAcetate=1)

        # Products
        glacial_acetic_acid = bst.Stream(ID='glacial_acetic_acid')
        wastewater = bst.Stream(ID='wastewater')

        # Recycles
        solvent_recycle = bst.Stream('solvent_rich')
        water_rich = bst.Stream('water_rich')
        distillate = bst.Stream('raffinate_distillate')
        
        
        # System and unit operations
        with bst.System('AAsep') as sys:
            extractor = bst.MultiStageMixerSettlers(
                'extractor',
                ins=(acetic_acid_broth, ethyl_acetate, solvent_recycle),
                outs=('extract', 'raffinate'),
                top_chemical='EthylAcetate',
                feed_stages=(0, -1, -1),
                N_stages=_n1,
                use_cache=True,
            )

            @extractor.add_specification(run=True)
            def adjust_fresh_solvent_flow_rate():
                broth = acetic_acid_broth.F_mass
                EtAc_recycle = solvent_recycle.imass['EthylAcetate']
                EtAc_required = broth * solvent_feed_ratio
                if EtAc_required < EtAc_recycle:
                    solvent_recycle.F_mass *= EtAc_required / EtAc_recycle
                    EtAc_recycle = solvent_recycle.imass['EthylAcetate']
                EtAc_fresh = EtAc_required - EtAc_recycle
                ethyl_acetate.imass['EthylAcetate'] = max(
                    0, EtAc_fresh
                )

            HX = bst.HXutility(
                'extract_heater',
                ins=(extractor.extract),
                outs=('hot_extract'),
                rigorous=True,
                V=0,
            )
            ED = bst.ShortcutColumn(
                'extract_distiller',
                ins=HX-0,
                outs=['', 'acetic_acid'],
                LHK=('Water', 'AceticAcid'),
                Lr=_Lr1,
                Hr=_Hr1,
                k=_k1,
                partial_condenser=False,
            )
            ED2 = bst.ShortcutColumn(
                'acetic_acid_purification',
                ins=ED-1,
                outs=('', glacial_acetic_acid),
                LHK=('EthylAcetate', 'AceticAcid'),
                Lr=_Lr1,
                Hr=_Hr2,
                k=_k2,
                partial_condenser=False
            )
            ED.check_LHK = ED2.check_LHK = False
            mixer = bst.Mixer(
                ins=(ED-0, ED2-0, distillate)
            )
            HX = bst.HXutility(ins=mixer-0, T=_T_hex)
            settler = bst.MixerSettler(
                'settler',
                ins=HX-0,
                outs=(solvent_recycle, water_rich),
                top_chemical='EthylAcetate',
            )
            mixer = bst.Mixer(ins=[extractor.raffinate, water_rich])
            RD = bst.ShortcutColumn(
                'raffinate_distiller',
                LHK=('EthylAcetate', 'Water'),
                ins=mixer-0,
                outs=[distillate, wastewater],
                partial_condenser=False,
                Lr=_Lr3,
                Hr=_Hr3,
                k=_k3,
            )
            
        sys.operating_hours = 330 * 24 # annual operating hours, hr/yr
        
        self.sys = sys

    def capex(self):
        # capex of equipment in MMUSD/yr
        capex = round(self.sys.installed_equipment_cost / 1e6, 4)
        
        try:
            int(capex) # checks if nan or a number is returned
            return capex
        
        except:
            return np.inf
        
    def opex(self):
        # opex of equipment in MMUSD/yr
        opex = round(self.sys.material_cost + self.sys.utility_cost / 1e6, 4)
    
        try:
            int(opex) # checks if nan or a number is returned
            return opex
        
        except:
            return np.inf
        
    def cost(self):
        return self.capex() + self.opex()
    
    def revenue(self):
        # price taken from website
        # https://businessanalytiq.com/procurementanalytics/index/acetic-acid-price-index/
        
        if self.acetic_acid_constraint() == 0: # return profit if in-specification
            stream = [stream for stream in self.sys.streams if stream.ID == 'glacial_acetic_acid'][0]
            P_AceticAcid = 0.4 # $/kg
            F_AceticAcid = stream.F_mass
            return round(P_AceticAcid * F_AceticAcid * self.sys.operating_hours / 1e6, 4) # Units: $/yr
        
        else: # return nothing if not in-specification
            return 0
    
    def profit(self):
        return self.revenue() - self.cost()
            
    def MSP(self):
        stream = [stream for stream in self.sys.streams if stream.ID == 'glacial_acetic_acid'][0]
        P_AceticAcid = 0.4 # $/kg
        F_AceticAcid = stream.F_mass * self.sys.operating_hours / 1e6 # kg/yr
        if self.acetic_acid_constraint() == 0: # return profit if in-specification
            return np.inf
        else:
            msp = self.cost() / F_AceticAcid # UNits: $/kg
            return round(msp, 4) 
        
    def simulate(self):
        self.nEval += 1
        self.sys.simulate()

    def wt_acetic_acid(self):
        stream = [stream for stream in self.sys.streams if stream.ID == 'glacial_acetic_acid'][0]
        return stream.get_mass_fraction(IDs='AceticAcid')
        
    def acetic_acid_constraint(self):
        x_desired = 0.98 # wt%
        x_achieved = self.wt_acetic_acid()
        d_x = x_desired - x_achieved # -ve if  product is in-spec
        
        cons = max(0, d_x) # return 0 if happy or constraint violation if not
        return cons
        
    def func(self, X=None):
        
        try:
            if X is None:
                self.set_base_model()
                
            else:# set and run the simulation
                self._set(X) # set the new operating parameters
                    
            self.simulate() # run the simulation
            
            # assess plant financials
            objective_function = self.MSP()
            
        # if failure for any reason, then reutrn a value of np.inf
        except:
            return np.inf


        if self.verbose:
            print(self.nEval, objective_function)
            
        return objective_function
    
    def natural_units(self, X):
        X_natural = np.zeros((np.shape(X)))
        bounds = self._bounds()
        b = np.array(bounds)
        d_b = b[:,1] - b[:,0] # range of bounds in natural units
        
        for i, x in enumerate(X):
            X_natural[i] = b[:,0] + (x[:] * d_b)
        
        return X_natural
            
    def optimize(self, method='DE'):
        
        # simple differential evolution approach based on scipy implementation 
        if method == 'DE':
            bounds = self._bounds()
            sampler = qmc.LatinHypercube(d=len(bounds))
            sample = sampler.random(n=50) # n is the population size!
            population = self.natural_units(sample)
            return differential_evolution(self.func, bounds=self._bounds(),
                                          integrality=self._integrality(),
                                          init=sample
                                          )

In [ ]:
bb = BlackBox(verbose=True)
results = bb.optimize()
